In [42]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

print ("Reading dataset...")
#Read the train dataset
dataset = pd.read_csv('../../input/trainv3.csv')
#Read test dataset
dataset_test = pd.read_csv('../../input/testv3.csv')

print ("Length of dataset = " + str(len(dataset)))

## cat117 is cont 2
## cat 118 is High vs. Low
print ("Factorizing categorical variables...")
features = dataset.columns
cats = [feat for feat in features if 'cat' in feat]
## print cats
for feat in cats:
    dataset[feat] = pd.factorize(dataset[feat], sort=True)[0]
    dataset_test[feat] = pd.factorize(dataset_test[feat], sort=True)[0]
print ("Finished loading and factorized data.")

Reading dataset...
Length of dataset = 188318
Factorizing categorical variables...
Finished loading and factorized data.


In [25]:
print ("Preprocessing the Data...")
## Response
shift = 200
response = np.log(dataset['loss'].values + shift)

ids_train = dataset['id']

## Drop the response from our dataset
dataset = dataset.drop(['loss', 'id'], 1)

print "Responses:"
print response

Preprocessing the Data...
Responses:
[ 7.78870066  7.30222685  8.07249545 ...,  8.69326862  7.47469844
  8.50749027]


In [66]:
print("-")*50
print "Random Forest Algo with KFold"
print("-")*50
#Evaluation of various combinations of RandomForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import cross_validation
from sklearn.cross_validation import KFold

print ("Creating Training and Validation sets...")
#get the number of rows and columns

seed = 0

## CODE FOR 1-FOLD VALIDATION
## FOR REFERENCE:
##X_train is 90% training features
##Y_train is 90% training responses

##X_val is the 10% validation features
##Y_val is the 10% validation responses

#X_train, X_val, Y_train, Y_val = cross_validation.train_test_split(dataset, response, test_size=.1, random_state=seed)
#print "Length of Validation Set:" + str(len(Y_val))
#print "Length of Train Set:" + str(len(Y_train))
#print "Ratio: " + str(len(Y_val) / float(len(Y_train)))


## K-Fold Validation
print ("Creating K-fold validation dataset indices")
n_folds = 10
kf = KFold(dataset.shape[0], n_folds=n_folds, shuffle = True)
pred_test = 0
temp_cv_score = []
cv_loss = pd.DataFrame(columns=["id","loss"])

for i, (train_index, test_index) in enumerate(kf):
    print "-" * 80
    print('\nFold %d' % (i + 1))

    X_train, X_val = dataset.iloc[train_index], dataset.iloc[test_index]
    Y_train, Y_val = response[train_index], response[test_index]
    print "Training size: " + str(len(X_train))
    print "Validation size: " + str(len(X_val))
    print "Total size: " + str(len(X_train) + len(X_val))

    
    seed = 0
    print "Training Random Forest Model..."
    n_estimators = 50
    ## n_jobs=-1: Use all cores
    ## n_estimators: Create random forest of 200 trees
    model = RandomForestRegressor(n_jobs=-1,
                                  n_estimators=n_estimators,
                                  max_features="sqrt",
                                  oob_score = True,
                                  verbose=1,
                                  random_state=seed)
    model.fit(X_train, Y_train)
    importances = model.feature_importances_
    
    results = np.exp(model.predict(X_val)) - 200
    cv_loss = pd.concat([cv_loss, pd.DataFrame({"id": ids_train[test_index], "loss": results})])
    
    MAE = mean_absolute_error(np.exp(Y_val) - shift, np.exp(model.predict(X_val)) - shift)
    print "MAE: " + str(MAE)
    
    pred_test += np.exp(model.predict(dataset_test.drop(['id'],1))) - shift

# filename = str(n_folds) + "Fold_" + str(n_estimators) + "Forest_CV_losses.csv"
# cv_loss.to_csv(filename, index=False, sort=True)

# submission = pd.DataFrame()
# submission['id'] = dataset_test['id']
# submission['loss'] = pred_test/n_folds
# filename = str(n_folds) + "Fold_" + str(n_estimators) + "Forest_Predictions_losses.csv"
# submission.to_csv(filename, index=False, sort=True)


--------------------------------------------------
Random Forest Algo with KFold
--------------------------------------------------
Creating Training and Validation sets...
Creating K-fold validation dataset indices
--------------------------------------------------------------------------------

Fold 1
Training size: 169486
Validation size: 18832
Total size: 188318
Training Random Forest Model...


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   20.9s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.3s finished


MAE: 407.787791607


ValueError: Number of features of the model must  match the input. Model n_features is 132 and  input n_features is 130 

In [65]:
cv_50 = pd.read_csv('10Fold_50Forest_CV_losses.csv')
cv_200 = pd.read_csv('10Fold_200Forest_CV_losses.csv')



mean_absolute_error(np.array(dataset['loss']), np.array(cv_50.sort(['id'], ascending=1)['loss']))

1212.2262741234279

In [17]:
filename = str(n_folds) + "Fold_" + str(n_estimators) + "Forest_CV_losses.csv"
cv_loss.to_csv(filename, index=False, sort=True)

submission = pd.DataFrame()
submission['id'] = dataset_test['id']
submission['loss'] = pred_test/n_folds
filename = str(n_folds) + "Fold_" + str(n_estimators) + "Forest_Predictions_losses.csv"
submission.to_csv(filename, index=False, sort=True)

In [39]:
print("-")*50
print "Random Forest Algo with 1 Fold Validation"
print("-")*50
#Evaluation of various combinations of RandomForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import cross_validation
from sklearn.cross_validation import KFold

print ("Creating Training and Validation sets...")
#get the number of rows and columns

seed = 0

## CODE FOR 1-FOLD VALIDATION
## FOR REFERENCE:
#X_train is 90% training features
#Y_train is 90% training responses

#X_val is the 10% validation features
#Y_val is the 10% validation responses

X_train, X_val, Y_train, Y_val = cross_validation.train_test_split(dataset, response, test_size=.1, random_state=seed)
print "Length of Validation Set:" + str(len(Y_val))
print "Length of Train Set:" + str(len(Y_train))
print "Ratio: " + str(len(Y_val) / float(len(Y_train)))

n_list = np.array([50])

for i in n_list:
    seed = 0
    print "Training Random Forest Model..."
    ## n_jobs=-1: Use all cores
    ## n_estimators: Create random forest of 50 trees
    model = RandomForestRegressor(n_jobs=-1,n_estimators=i,random_state=seed)
    model.fit(X_train, Y_train)
    
    results = np.exp(model.predict(X_val)) - 200
    print pd.DataFrame(results)
    # pd.DataFrame(results).to_csv("results.csv", index=False)

    result = i, mean_absolute_error(np.exp(Y_val), np.exp(model.predict(X_val)))
    print result
    
predictions = np.exp(model.predict(dataset_test.drop(['id'],1))) - shift
final = pd.DataFrame({"id": dataset_test['id'], "loss": predictions})
print final
final.to_csv("predictions.csv", index=False)



--------------------------------------------------
Random Forest Algo with 1 Fold Validation
--------------------------------------------------
Creating Training and Validation sets...
Length of Validation Set:18832
Length of Train Set:169486
Ratio: 0.111112422265
Training Random Forest Model...


KeyboardInterrupt: 

In [38]:
## Code for Tuning number of trees

print("-")*50
print "Random Forest Algo with 1 Fold Validation"
print("-")*50
#Evaluation of various combinations of RandomForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import cross_validation
from sklearn.cross_validation import KFold

print ("Creating Training and Validation sets...")
#get the number of rows and columns

seed = 0

## CODE FOR 1-FOLD VALIDATION
## FOR REFERENCE:
#X_train is 90% training features
#Y_train is 90% training responses

#X_val is the 10% validation features
#Y_val is the 10% validation responses

X_train, X_val, Y_train, Y_val = cross_validation.train_test_split(dataset, response, test_size=.1, random_state=seed)
print "Length of Validation Set:" + str(len(Y_val))
print "Length of Train Set:" + str(len(Y_train))
print "Ratio: " + str(len(Y_val) / float(len(Y_train)))

MAE = []
n_list = np.array([1,5,10,20,30,40,50,60,70,80,90,100,200,300])

for i in n_list:
    seed = 0
    print "Training Random Forest Model..."
    print str(i) + " Number of Trees..."
    ## n_jobs=-1: Use all cores
    ## n_estimators: Create random forest of 50 trees
    model = RandomForestRegressor(n_jobs=-1,
                                  n_estimators=i,
                                  max_features="sqrt",
                                  oob_score = True,
                                  verbose=1,
                                  random_state=seed)
    model.fit(X_train, Y_train)
    
    results = np.exp(model.predict(X_val)) - 200

    result = i, mean_absolute_error(np.exp(Y_val), np.exp(model.predict(X_val)))
    MAE.append(result)
    print MAE

pd.DataFrame(MAE).to_csv("MAE.csv", index=False)
    



--------------------------------------------------
Random Forest Algo with 1 Fold Validation
--------------------------------------------------
Creating Training and Validation sets...
Length of Validation Set:18832
Length of Train Set:169486
Ratio: 0.111112422265
Training Random Forest Model...
1 Number of Trees...


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


              0
0       5293.62
1       1263.19
2       4570.22
3       1438.98
4       1505.39
5       5376.31
6       2527.29
7        343.53
8       1623.05
9       3173.48
10      2058.37
11       168.28
12      2963.10
13      6587.07
14      3601.19
15      1189.41
16      4925.34
17      3767.13
18      2447.72
19      3128.75
20      2027.65
21      1072.48
22      3140.14
23     10427.95
24      4502.35
25      1322.84
26       364.92
27      1427.01
28      2842.34
29      1527.94
...         ...
18802   1299.66
18803   2376.21
18804   1891.40
18805   3069.22
18806   9344.65
18807   2989.34
18808   2085.28
18809   2271.42
18810    760.10
18811   6189.07
18812   5118.88
18813   1574.08
18814   4174.21
18815   6491.26
18816   1247.00
18817   3553.49
18818   2520.79
18819   2365.29
18820   5287.48
18821   6020.20
18822   3148.59
18823    556.04
18824   1296.57
18825   4618.31
18826   1548.70
18827   7715.66
18828   1340.80
18829   2341.59
18830   1368.48
18831  17057.69

[18832 

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    0.0s finished


                 0
0      2326.235473
1      1559.673642
2      2572.284275
3      1142.524491
4      1224.885503
5      5669.638126
6      2574.036354
7       693.016659
8      2897.055815
9      1970.589251
10     1317.997208
11     1197.492619
12     3293.614112
13     5273.170369
14     5757.628589
15     1851.396448
16     5410.808749
17     4165.798993
18     1599.075079
19     4662.604408
20     4657.233118
21     2305.539100
22     5420.014881
23     3459.915186
24     2423.381073
25     5508.319311
26      769.183051
27     4956.997442
28      826.651042
29     1753.985892
...            ...
18802  1316.278812
18803  1416.284184
18804  1717.866919
18805  1658.138829
18806  3225.714700
18807  2746.579223
18808  1309.540531
18809  2550.842234
18810  1876.732015
18811  4707.850717
18812  3076.585430
18813   964.364898
18814  4581.998676
18815  2581.876240
18816  2290.888197
18817  2904.468890
18818  4562.954669
18819  2924.214820
18820  3924.946055
18821  2741.374813
18822  4238.

...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.8min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    4.1s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    6.3s finished


                 0
0      1941.420347
1      1691.079461
2      2454.234295
3       876.724950
4      1447.007655
5      6557.705149
6      2208.362830
7       690.284767
8      1684.471457
9      1923.525702
10     1452.560624
11     1638.057969
12     2823.575356
13     4875.581306
14     4109.190970
15     2154.650390
16     4359.920074
17     3694.189355
18     1346.893955
19     3388.073801
20     4497.017327
21     2514.434282
22     3966.317647
23     3028.103837
24     1504.595061
25     3032.022506
26      787.486694
27     4715.149150
28     1393.551516
29     1612.452515
...            ...
18802  1841.502809
18803  1197.691072
18804  1807.892918
18805  1804.012688
18806  3726.871683
18807  1589.677336
18808  1629.929283
18809  2924.494527
18810  2068.124556
18811  3051.511748
18812  2711.005729
18813  1435.427490
18814  4901.144229
18815  2473.117232
18816  1977.622081
18817  2697.618113
18818  2934.989116
18819  3102.609460
18820  3138.825655
18821  2668.611036
18822  4130.

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    4.0s


[(1, 1852.3735536620445), (5, 1328.7001159845779), (10, 1269.4253326089677), (20, 1232.4228383456134), (30, 1223.7479485529041), (40, 1218.9802203917311), (50, 1216.5838093306693), (60, 1213.693331209957), (70, 1211.9008216961377), (80, 1210.8823532415558), (90, 1210.3924708584934), (100, 1209.1758397649126), (200, 1206.6705722102038), (300, 1205.305347249322)]
(300, 1205.305347249322)


[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    6.2s finished


In [43]:


model.predict(X_val)

result = mean_absolute_error(np.exp(Y_val), np.exp(model.predict(X_val)))
print result


1199.52252619


In [39]:
print(np.max(np.exp(Y_val)))
print(np.max(np.exp(model.predict(X_val))))

print(np.min(np.exp(Y_val)))
print(np.min(np.exp(model.predict(X_val))))

79823.52
74284.6499438
208.4
212.468673402


In [45]:
predictions = np.exp(model.predict(dataset_test.drop(['id'],1))) - shift
pd.DataFrame(test_results).to_csv("test_results.csv", index=False)

In [46]:
predictions = np.exp(model.predict(dataset_test.drop(['id'],1))) - shift
final = pd.DataFrame({"id": dataset_test['id'], "loss": predictions})
print final
final.to_csv("predictions.csv", index=False)

            id          loss
0            4   1564.566853
1            6   1867.538184
2            9  17609.892638
3           12   8705.106719
4           15    667.158580
5           17   4467.462705
6           21   1587.517670
7           28   1082.680701
8           32   3196.317715
9           43   8423.991231
10          46   3785.479402
11          50   1257.128050
12          54   1015.295290
13          62   1819.752481
14          70   2082.138470
15          71   9916.939520
16          75   3882.482637
17          77   2714.188691
18          81   5479.230856
19          83   2339.194765
20          87   2300.101534
21          97   1769.455781
22         103   1392.603501
23         119   1318.940507
24         120   1837.763600
25         127   1138.647078
26         138   3378.242777
27         141   2632.653634
28         148    952.278967
29         150   3253.852208
...        ...           ...
125516  587482   1396.866419
125517  587484   4719.304256
125518  587489